In [567]:
import random
import utils
from math import ceil

In [568]:
primeArray = utils.read_data('primes.txt')

In [569]:
def Encrypt(message, n, e):
    cipher = ""
    noChar = utils.getNumberOfChars(n)
    maxCipherLength = len(str(n-1)) 
    for i in range(0, len(message), noChar):
        blockToEncrypt = utils.ConvertToInt(message[:noChar]) #indexing outofbounds works fine in python
        cipherdBlock = str(utils.PowerMod(blockToEncrypt, e, n))
        if (len(cipherdBlock) < maxCipherLength):  #make sure all cipher blocks are the same length
            cipherdBlock = cipherdBlock.zfill(maxCipherLength)
        cipher += cipherdBlock
        message = message[noChar:]
    return cipher


In [570]:
def Decrypt(cipher,n, d):
    maxCipherLength = len(str(n-1)) 
    message = ""
    for i in range(0, len(cipher), maxCipherLength):
        blockToDecrypt = int(cipher[:maxCipherLength])
        message += utils.ConvertToStr(utils.PowerMod(blockToDecrypt, d, n))
        cipher = cipher[maxCipherLength:]
    return message

In [571]:

#implement RSA Algorithm to generate public and private key
def RSA_key_generator(p,q):
    n = p*q
    phi = (p-1)*(q-1)
    e = random.randint(2,phi-1)     #choose e randomly from 1 to phi
    while utils.GCD(e,phi) != 1:
        e = random.randint(2,phi-1)
    d = utils.InverseModulo(e,phi)
    return (e,d,n)

In [572]:
#generate two random integers from the array of prime numbers
def generate_two_prime_numbers(prime_array):
    p = random.choice(prime_array)
    q = random.choice(prime_array)
    while p == q:
        q = random.choice(prime_array)
    return (p,q)

In [574]:
#genertate two random integers from the array of prime numbers
(p,q) = generate_two_prime_numbers(primeArray)

#generate public and private key
(e,d,n) = RSA_key_generator(p,q)



message = "it fucking works and i am so happy"



#encrypt the message
cypher = Encrypt(message,n,d)



#decrypt the message
decrypted_message = Decrypt(cypher,n,e)


#print the  decrypted message

print(decrypted_message)


it fucking works and i am so happy
